In [4]:
import numpy as np

# Example daily returns (in decimal form) for a sample strategy
daily_returns = np.array([0.002, -0.001, 0.003, 0.0005, 0.0025, -0.002, 0.001])

# Assume an annual risk-free rate of 2% and convert it to a daily rate
risk_free_rate_annual = 0.01  
risk_free_rate_daily = risk_free_rate_annual / 252  # ~252 trading days in a year

# Calculate excess returns by subtracting the risk-free rate from the daily returns
excess_returns = daily_returns - risk_free_rate_daily

# Compute the mean and standard deviation of the excess returns
mean_excess = np.mean(excess_returns)
std_excess = np.std(excess_returns)

# Calculate the daily Sharpe Ratio and annualize it (multiply by sqrt(252))
sharpe_ratio = (mean_excess / std_excess) * np.sqrt(252)

print("Mean Excess Return:", mean_excess)
print("Standard Deviation of Excess Return:", std_excess)
print("Annualized Sharpe Ratio:", sharpe_ratio)


Mean Excess Return: 0.0008174603174603175
Standard Deviation of Excess Return: 0.0017053337694733315
Annualized Sharpe Ratio: 7.609525168783332


In [5]:
import numpy as np

# Example daily returns in decimal form (e.g., 0.01 for 1% return)
daily_returns = np.array([0.001, -0.002, 0.003, 0.0005, 0.002, -0.001, 0.004])

# Calculate the cumulative return over the period
# (Product of (1 + daily return) for each day) - 1 gives total return
cumulative_return = np.prod(1 + daily_returns) - 1

# Number of trading days in the period
n = len(daily_returns)

# Annualize the cumulative return:
# Formula: Annualized Return = (1 + cumulative_return)^(252/n) - 1
annualized_return = (1 + cumulative_return) ** (252 / n) - 1

print("Cumulative Return over period:", cumulative_return)
print("Annualized Return:", annualized_return)

Cumulative Return over period: 0.007510468426500161
Annualized Return: 0.30913496910123484


In [6]:
import pandas as pd
import numpy as np

# Mock data
data = {
    "Date": ["2023-01-01", "2023-01-02", "2023-01-03", "2023-01-04", "2023-01-05"],
    "Close": [100, 102, 98, 101, 103],
    "Sentiment": [0.8, -0.5, 0.2, -0.3, 0.6],
    "Actual_Label": [1, 1, 0, 0, 1],  # 1=Up, 0=Down
    "Predicted_Label": [1, 0, 0, 1, 1],  # Model predictions
    "Predicted_Probability": [0.9, 0.4, 0.3, 0.6, 0.8],  # Confidence scores
    "Actual_Return": [0.02, -0.01, -0.03, 0.015, 0.025]  # Daily returns
}

df = pd.DataFrame(data)
print(df)

         Date  Close  Sentiment  Actual_Label  Predicted_Label  \
0  2023-01-01    100        0.8             1                1   
1  2023-01-02    102       -0.5             1                0   
2  2023-01-03     98        0.2             0                0   
3  2023-01-04    101       -0.3             0                1   
4  2023-01-05    103        0.6             1                1   

   Predicted_Probability  Actual_Return  
0                    0.9          0.020  
1                    0.4         -0.010  
2                    0.3         -0.030  
3                    0.6          0.015  
4                    0.8          0.025  


In [7]:
# Calculate strategy returns (1% transaction cost assumed)
df["Strategy_Return"] = np.where(
    df["Predicted_Label"] == df["Actual_Label"],
    df["Actual_Return"] - 0.01,  # Win with cost
    -abs(df["Actual_Return"]) - 0.01  # Loss with cost
)

# Annualized Sharpe Ratio
returns = df["Strategy_Return"]
sharpe_ratio = (returns.mean() * 252) / (returns.std() * np.sqrt(252))
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

Sharpe Ratio: -8.07


In [8]:
# Calculate profit factor
gross_profit = df[df["Strategy_Return"] > 0]["Strategy_Return"].sum()
gross_loss = abs(df[df["Strategy_Return"] < 0]["Strategy_Return"].sum())
profit_factor = gross_profit / gross_loss
print(f"Profit Factor: {profit_factor:.2f}")

Profit Factor: 0.29


In [9]:
# Calculate cumulative returns
cumulative_returns = (1 + returns).cumprod()
peak = cumulative_returns.expanding().max()
drawdown = (cumulative_returns - peak) / peak
max_drawdown = drawdown.min()
print(f"Max Drawdown: {max_drawdown:.2%}")

Max Drawdown: -8.27%


In [10]:
# Calculate weighted returns
df["Weighted_Return"] = df["Predicted_Probability"] * df["Actual_Return"]
total_weighted_return = df["Weighted_Return"].sum()
print(f"Precision-Weighted Return: {total_weighted_return:.4f}")

Precision-Weighted Return: 0.0340


In [11]:
# Calculate monetary impact (assuming $10,000 position size)
position_size = 10000

tp = df[(df["Predicted_Label"] == 1) & (df["Actual_Label"] == 1)]["Actual_Return"].sum() * position_size
fp = df[(df["Predicted_Label"] == 1) & (df["Actual_Label"] == 0)]["Actual_Return"].sum() * position_size
fn = df[(df["Predicted_Label"] == 0) & (df["Actual_Label"] == 1)]["Actual_Return"].sum() * position_size
tn = abs(df[(df["Predicted_Label"] == 0) & (df["Actual_Label"] == 0)]["Actual_Return"].sum()) * position_size

print(f"""
Economic Confusion Matrix ($):
-----------------------------
|          | Predicted Up | Predicted Down |
|----------|--------------|-----------------|
| Actual Up| ${tp:,.0f}    | ${fn:,.0f} (Missed)|
| Actual Dn| ${fp:,.0f} (Loss)| ${tn:,.0f} (Saved) |
""")


Economic Confusion Matrix ($):
-----------------------------
|          | Predicted Up | Predicted Down |
|----------|--------------|-----------------|
| Actual Up| $450    | $-100 (Missed)|
| Actual Dn| $150 (Loss)| $300 (Saved) |



In [12]:
def adjusted_win_rate(df, commission=0.0001):
    trades = df[df["Predicted_Label"] != 0]  # Count only active trades
    gross_profit = trades[trades["Strategy_Return"] > 0]["Strategy_Return"].sum()
    costs = len(trades) * commission
    return (gross_profit - costs) / trades["Strategy_Return"].abs().sum()

win_rate = adjusted_win_rate(df)
print(f"Turnover-Adjusted Win Rate: {win_rate:.2%}")

Turnover-Adjusted Win Rate: 49.40%


In [13]:
# Assume benchmark returns (e.g., S&P 500)
benchmark_returns = np.array([0.01, 0.005, -0.02, 0.015, 0.01])  # Example values
alpha = returns.mean() - benchmark_returns.mean()
print(f"Alpha vs Benchmark: {alpha:.4f}")

Alpha vs Benchmark: -0.0160


In [1]:
import torch

# Example text (news headline) and numeric features for a single data point
text = "stocks rally after record earnings report"
# A very simple tokenization for illustration:
vocab = {"stocks": 0, "rally": 1, "after": 2, "record": 3, "earnings": 4, "report": 5}
text_tokens = [vocab[word] for word in text.split()]
# Numeric features: e.g., [daily_percent_change, volume_zscore, volatility]
numeric_features = [0.05, 1.2, 0.3]  # dummy values

# Convert to tensors
text_indices = torch.tensor([text_tokens])          # shape: (batch=1, seq_len)
numeric_tensor = torch.tensor([numeric_features])   # shape: (batch=1, num_features)
print("Text indices:", text_indices)
print("Numeric tensor:", numeric_tensor)


Text indices: tensor([[0, 1, 2, 3, 4, 5]])
Numeric tensor: tensor([[0.0500, 1.2000, 0.3000]])


In [2]:
import torch.nn as nn

embed_dim = 16
vocab_size = len(vocab)
num_features = numeric_tensor.shape[1]

# Embedding layer for text tokens and linear layer for numeric features
text_embed_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)
numeric_embed_layer = nn.Linear(num_features, embed_dim)

# Apply the embedding layers to the inputs
text_embedded = text_embed_layer(text_indices)      # shape: (1, seq_len, embed_dim)
numeric_embedded = numeric_embed_layer(numeric_tensor)  # shape: (1, embed_dim)

print("Text embedding shape:", text_embedded.shape)
print("Numeric embedding shape:", numeric_embedded.shape)


Text embedding shape: torch.Size([1, 6, 16])
Numeric embedding shape: torch.Size([1, 16])


In [3]:
# Transpose text to (seq_len, batch, embed_dim)
text_embed_seq = text_embedded.permute(1, 0, 2)        # shape: (seq_len, 1, 16)
# Add a seq_len dimension of 1 to numeric embed and transpose to (1, batch, embed_dim)
numeric_embed_seq = numeric_embedded.unsqueeze(0).permute(0, 1, 2)  # shape: (1, 1, 16)

print("Text sequence shape (L,N,E):", text_embed_seq.shape)
print("Numeric sequence shape (L,N,E):", numeric_embed_seq.shape)


Text sequence shape (L,N,E): torch.Size([6, 1, 16])
Numeric sequence shape (L,N,E): torch.Size([1, 1, 16])


In [4]:
# Initialize a MultiheadAttention module
d_model = embed_dim  # embedding dimension
num_heads = 1        # single-head attention for simplicity
cross_attn = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, batch_first=False)

# Perform cross-modal attention: numeric queries attend to textual keys/values
attended_numeric, attn_weights = cross_attn(query=numeric_embed_seq, key=text_embed_seq, value=text_embed_seq)

print("Attended numeric shape:", attended_numeric.shape)
print("Attention weights shape:", attn_weights.shape)


Attended numeric shape: torch.Size([1, 1, 16])
Attention weights shape: torch.Size([1, 1, 6])


In [5]:
# Combine the attended numeric embedding with original numeric embedding
attended_numeric_vec = attended_numeric.permute(1, 0, 2).reshape(1, -1)  # (1, 16)
original_numeric_vec = numeric_embedded  # (1, 16), already batch x embed
fused_vector = torch.cat([attended_numeric_vec, original_numeric_vec], dim=1)  # shape: (1, 32)

# A simple linear classifier on top of the fused features
output_layer = nn.Linear(32, 2)  # e.g., two output classes (rise vs fall)
prediction = output_layer(fused_vector)

print("Fused feature shape:", fused_vector.shape)
print("Output prediction shape:", prediction.shape)


Fused feature shape: torch.Size([1, 32])
Output prediction shape: torch.Size([1, 2])


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Example dimensions
batch_size = 32
num_numeric_features = 10
text_seq_len = 50
embed_dim = 128

# Dummy inputs (replace with real data in practice)
numeric_input = torch.randn(batch_size, num_numeric_features)       # [batch_size, num_numeric_features]
text_embeddings = torch.randn(text_seq_len, batch_size, embed_dim)  # [text_seq_len, batch_size, embed_dim]

# Linear projection for numeric features to match embed_dim
numeric_proj = nn.Linear(num_numeric_features, embed_dim)
numeric_embed = F.relu(numeric_proj(numeric_input))      # [batch_size, embed_dim]
numeric_query = numeric_embed.unsqueeze(0)               # [1, batch_size, embed_dim] -> query sequence of length 1


In [7]:
# Initialize multi-head attention (e.g., 4 heads)
num_heads = 4
cross_attn = nn.MultiheadAttention(embed_dim, num_heads)

# Perform cross-attention: Query=numeric (1 x B x D), Key=Value=text (T x B x D)
attn_output, attn_weights = cross_attn(query=numeric_query, key=text_embeddings, value=text_embeddings)
# attn_output: [1, batch_size, embed_dim]
# attn_weights: [batch_size, 1, text_seq_len] (attention scores over text tokens for each query element)

# Remove the sequence length dimension (since it's 1) to get fused feature vector per sample
fused_vector = attn_output.squeeze(0)  # [batch_size, embed_dim]


In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Load ModernBERT (Base version)
model_id = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)

# Input text with a masked token
text = "The capital of France is [MASK]."
inputs = tokenizer(text, return_tensors="pt")

# Get model outputs
outputs = model(**inputs)

# Locate the position of the [MASK] token
mask_index = inputs["input_ids"][0].tolist().index(tokenizer.mask_token_id)

# Predict the masked token
predicted_id = outputs.logits[0, mask_index].argmax(axis=-1)
predicted_token = tokenizer.decode(predicted_id)

print("Predicted token:", predicted_token)


Predicted token:  Paris
